In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from scripts.utils import reduce_mem_usage
from scripts.anomaly import anomaly_detector
from sklearn.model_selection import StratifiedKFold
import h5py
import ghalton
from scipy.stats import rankdata
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import pickle

NO_WEATHER = True

***

function to perform correction of weather data to localtime

In [2]:
sites_time_corrections = {0:4, 1:0, 2:7, 3:4, 4:7, 5:0, 6:4, 7:4, 8:4,
                          9:5, 10:7, 11:4, 12:0, 13:5, 14:4, 15:4}

def set_localtime(data):
    data = data.copy(deep=True)
    min_timestamp = data.timestamp.min()
    for site_id, hdiff in sites_time_corrections.items():
        idx = data.query("site_id == @site_id").index 
        data.loc[idx, "timestamp"] = data.loc[idx, "timestamp"] - pd.offsets.Hour(hdiff)
    return data.query("timestamp >= @min_timestamp")

function to fill missing values in weather dataframes

In [3]:
def weather_imputation(weather_data):
    weather_data = (weather_data
        .set_index("timestamp")
        .groupby('site_id')
        .apply(lambda group: group.interpolate(method="time", limit_direction='both'))
        .reset_index())
    return weather_data

***

In [2]:
train = pd.read_csv("data/train.csv", parse_dates=["timestamp"])

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


***

In [4]:
building_metadata = pd.read_csv("data/building_metadata.csv")

In [5]:
building_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null object
square_feet    1449 non-null int64
year_built     675 non-null float64
floor_count    355 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB


In [6]:
print(f"Missing values in year_built: {np.sum(building_metadata.year_built.isna())}")

Missing values in year_built: 774


In [7]:
print(f"Missing values in floor_count: {np.sum(building_metadata.floor_count.isna())}")

Missing values in floor_count: 1094


In [8]:
building_metadata.drop(["year_built","floor_count"], axis=1, inplace=True)

***

In [28]:
weather_train = pd.read_csv("data/weather_train.csv", parse_dates=["timestamp"])

In [29]:
weather_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null datetime64[ns]
air_temperature       139718 non-null float64
cloud_coverage        70600 non-null float64
dew_temperature       139660 non-null float64
precip_depth_1_hr     89484 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 9.6 MB


In [30]:
100*weather_train.isna().sum()/weather_train.shape[0]

site_id                0.000000
timestamp              0.000000
air_temperature        0.039350
cloud_coverage        49.489529
dew_temperature        0.080845
precip_depth_1_hr     35.979052
sea_level_pressure     7.596603
wind_direction         4.484414
wind_speed             0.217496
dtype: float64

In [31]:
weather_train.groupby('site_id').apply(lambda group: group.isna().sum())

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,,,
0,0,0,3,3830,3,1,85,250,0
1,0,0,1,7062,1,8763,52,3,0
2,0,0,1,2354,1,66,44,591,5
3,0,0,4,3642,6,37,158,153,4
4,0,0,0,4230,2,1317,73,105,0
5,0,0,2,6030,2,8755,8755,295,3
6,0,0,10,2992,10,0,175,779,39
7,0,0,0,8614,23,7883,8,0,0
8,0,0,3,3830,3,1,85,250,0


In [32]:
weather_train.groupby('site_id').apply(lambda group: 100*group.isna().sum()/group.shape[0])

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,,,
0,0.0,0.0,0.034153,43.602004,0.034153,0.011384,0.967668,2.846084,0.000000
1,0.0,0.0,0.011412,80.588839,0.011412,100.000000,0.593404,0.034235,0.000000
2,0.0,0.0,0.011386,26.801776,0.011386,0.751452,0.500968,6.728908,0.056928
3,0.0,0.0,0.045558,41.480638,0.068337,0.421412,1.799544,1.742597,0.045558
4,0.0,0.0,0.000000,48.161221,0.022771,14.994876,0.831151,1.195491,0.000000
5,0.0,0.0,0.022844,68.874929,0.022844,100.000000,100.000000,3.369503,0.034266
6,0.0,0.0,0.113869,34.069688,0.113869,0.000000,1.992712,8.870417,0.444090
7,0.0,0.0,0.000000,100.000000,0.267007,91.513815,0.092872,0.000000,0.000000
8,0.0,0.0,0.034153,43.602004,0.034153,0.011384,0.967668,2.846084,0.000000


In [33]:
weather_train.drop(["cloud_coverage", "precip_depth_1_hr"], axis=1, inplace=True)

In [34]:
weather_train = weather_imputation(weather_train)

In [35]:
weather_train.groupby('site_id').apply(lambda group: group.isna().sum())

,timestamp,site_id,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,8755,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0


***
merges the datasets

In [13]:
if NO_WEATHER:
    train_data = pd.merge(train, building_metadata, how="left", on=["building_id"])
else:
    train_data = (pd.merge(train, building_metadata, how="left", on=["building_id"])
                  .merge(weather_train, how="left", on=["timestamp","site_id"]))

***
removes anomal behavior for `site_id=0`

In [14]:
train_data_cut = train_data.query("site_id == 0")
ts_uid_values = train_data_cut.loc[:, ["building_id", "meter"]].drop_duplicates()
delete_idx = pd.Index(np.array([], dtype=int))
for i,row in ts_uid_values.iterrows():
    ts = train_data_cut.query("building_id == @row.building_id & meter == @row.meter")
    if ts.query("timestamp < '2016-05-20 18:00:00'").meter_reading.quantile(0.75) == 0:
        delete_idx = delete_idx.union(ts.query("timestamp < '2016-05-20 18:00:00'").index)  
        
train_data.drop(delete_idx, axis=0, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19871167 entries, 0 to 19871166
Data columns (total 7 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
site_id          int64
primary_use      object
square_feet      int64
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 1.0+ GB


***
target correction for `site_id = 0`

* kBTU -> kWh (* 0.2931)

In [16]:
idx = train_data.query("site_id == 0 & meter == 0").index
train_data.loc[idx, "meter_reading"] = 0.2931*train_data.loc[idx, "meter_reading"]

***
removes anormal behavior for `building_id=363` before `2016-07-25`

In [17]:
idx = train_data.query("building_id == 363 & meter == 0 & timestamp <= '2016-07-24 23:00:00'").index
train_data.drop(idx, axis=0, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [18]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19866224 entries, 0 to 19866223
Data columns (total 7 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
site_id          int64
primary_use      object
square_feet      int64
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 1.0+ GB


***
### outlier removal

In [ ]:
ts_uid_values = train_data.loc[:, ["building_id","meter"]].drop_duplicates()
outliers_indexes = list()

for _,row in ts_uid_values.iterrows():
    ts = train_data.query("building_id == @row.building_id & meter == @row.meter")
    ts = ts.reset_index()
    outliers = anomaly_detector(ts.meter_reading, window_size=48, sigma=5)
    if len(outliers) > 0:
        print(f"{len(outliers)} found for building_id=={row.building_id} & meter={row.meter}")
    outliers_indexes.append(ts.loc[[idx for idx,_ in outliers], :].index.values)

***

In [19]:
train_data = reduce_mem_usage(train_data)

Mem. usage decreased to 530.49 Mb (50.0% reduction)


In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19866224 entries, 0 to 19866223
Data columns (total 7 columns):
building_id      int16
meter            int8
timestamp        datetime64[ns]
meter_reading    float32
site_id          int8
primary_use      object
square_feet      int32
dtypes: datetime64[ns](1), float32(1), int16(1), int32(1), int8(2), object(1)
memory usage: 530.5+ MB


In [21]:
if NO_WEATHER:
    train_data.to_hdf("data/train_data_nw.h5", key="train_data")
else:
    train_data.to_hdf("data/train_data.h5", key="train_data")

***

In [22]:
test = pd.read_csv("data/test.csv", parse_dates=["timestamp"])

In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.2 GB


***

In [36]:
weather_test = pd.read_csv("data/weather_test.csv", parse_dates=["timestamp"])

In [37]:
100*weather_test.isna().sum()/weather_test.shape[0]

site_id                0.000000
timestamp              0.000000
air_temperature        0.037512
cloud_coverage        50.658808
dew_temperature        0.117947
precip_depth_1_hr     34.478057
sea_level_pressure     7.670167
wind_direction         4.461790
wind_speed             0.165919
dtype: float64

In [39]:
weather_test.groupby('site_id').apply(lambda group: 100*group.isna().sum()/group.shape[0])

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,,,
0,0.0,0.0,0.000000,43.053653,0.000000,0.171233,1.763699,2.722603,0.011416
1,0.0,0.0,0.127263,79.094117,0.127263,100.000000,0.376005,0.028923,0.000000
2,0.0,0.0,0.000000,31.078767,0.011416,0.119863,0.188356,6.809361,0.051370
3,0.0,0.0,0.011417,44.819318,0.017126,0.251185,2.146486,1.438603,0.045670
4,0.0,0.0,0.000000,45.843800,0.017127,1.609957,0.679379,1.164649,0.000000
5,0.0,0.0,0.000000,65.802448,0.005800,100.000000,100.000000,3.103068,0.005800
6,0.0,0.0,0.022852,36.951554,0.022852,0.251371,2.222349,8.066728,0.291362
7,0.0,0.0,0.000000,100.000000,0.307729,89.157063,0.253424,0.000000,0.000000
8,0.0,0.0,0.000000,43.053653,0.000000,0.171233,1.763699,2.722603,0.011416


In [40]:
weather_test.drop(["cloud_coverage", "precip_depth_1_hr"], axis=1, inplace=True)

In [41]:
weather_test = weather_imputation(weather_test)

In [42]:
weather_test.groupby('site_id').apply(lambda group: 100*group.isna().sum()/group.shape[0])

,timestamp,site_id,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
site_id,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,100.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***

In [26]:
if NO_WEATHER:
    test_data = pd.merge(test, building_metadata, how="left", on=["building_id"])
else: 
    test_data = (pd.merge(test, building_metadata, how="left", on=["building_id"])
                 .merge(weather_test, how="left", on=["timestamp","site_id"]))

In [27]:
test_data = reduce_mem_usage(test_data)

Mem. usage decreased to 1431.57 Mb (43.8% reduction)


In [28]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 7 columns):
row_id         int32
building_id    int16
meter          int8
timestamp      datetime64[ns]
site_id        int8
primary_use    object
square_feet    int32
dtypes: datetime64[ns](1), int16(1), int32(2), int8(2), object(1)
memory usage: 1.4+ GB


In [29]:
if NO_WEATHER:
    test_data.to_hdf("data/test_data_nw.h5", key="test_data") 
else:
    test_data.to_hdf("data/test_data.h5", key="test_data")

In [30]:
test_data

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet
0,0,0,0,2017-01-01 00:00:00,0,Education,7432
1,1,1,0,2017-01-01 00:00:00,0,Education,2720
2,2,2,0,2017-01-01 00:00:00,0,Education,5376
3,3,3,0,2017-01-01 00:00:00,0,Education,23685
4,4,4,0,2017-01-01 00:00:00,0,Education,116607
...,...,...,...,...,...,...,...
41697595,41697595,1444,0,2018-05-09 07:00:00,15,Entertainment/public assembly,19619
41697596,41697596,1445,0,2018-05-09 07:00:00,15,Education,4298
41697597,41697597,1446,0,2018-05-09 07:00:00,15,Entertainment/public assembly,11265
41697598,41697598,1447,0,2018-05-09 07:00:00,15,Lodging/residential,29775


verify that row_id column is sorted in `test_data`

In [31]:
np.all(test_data.row_id.diff(1).fillna(1) == 1)

True

***
### validation data

In [29]:
splitter = StratifiedKFold(n_splits=4, shuffle=True, random_state=23)
valid_indexes = [valid_index for _,valid_index in splitter.split(train_data, train_data['building_id'])]

In [40]:
h5f = h5py.File("data/valid_sm_skfold_4fold_shuffle.h5", "w")
for i,valid_index in enumerate(valid_indexes):
    h5f.create_dataset(f'fold{i}', data=valid_indexes[i])
h5f.close()

***
### custom validation data - by week - with hausdorff sampling

In [30]:
n_folds = 4
_train_data = train_data.loc[:, ["building_id","meter","timestamp"]]
ts_uid_values = _train_data.loc[:, ["building_id","meter"]].drop_duplicates()

In [31]:
valid_indexes = [[] for i in range(n_folds)]
generator = ghalton.Halton(1)

for _,row in tqdm(ts_uid_values.iterrows()):
    ts = _train_data.query("building_id == @row.building_id & meter == @row.meter")
    ts["week"] = ts.timestamp.dt.week
    weeks = ts.week.unique()
    n_weeks = len(weeks)
    
    sequence = np.asarray(generator.get(n_weeks))[:,0]
    idx = rankdata(sequence).astype(int)-1

    for i,weeks_idx in enumerate(np.array_split(idx, n_folds)):
        weeks_by_fold = weeks[weeks_idx]
        valid_indexes[i].append(ts.query("week in @weeks_by_fold").index.values)

2380it [03:03, 12.95it/s]


In [33]:
h5f = h5py.File("data/valid_sm_custom_4fold.h5", "w")
for i,valid_index in enumerate(valid_indexes):
    h5f.create_dataset(f'fold{i}', data=np.concatenate(valid_index))
h5f.close()

***
### custom validation data - by week - with shuffle

In [45]:
np.random.seed(23)
n_folds = 4
_train_data = train_data.loc[:, ["building_id","meter","timestamp"]]
ts_uid_values = _train_data.loc[:, ["building_id","meter"]].drop_duplicates()

In [46]:
valid_indexes = [[] for i in range(n_folds)]

for _,row in tqdm(ts_uid_values.iterrows()):
    ts = _train_data.query("building_id == @row.building_id & meter == @row.meter")
    ts["week"] = ts.timestamp.dt.week
    weeks = ts.week.unique()
    np.random.shuffle(weeks)
    weeks_split = np.array_split(weeks, n_folds)
    np.random.shuffle(weeks_split)
    
    for i,weeks_by_fold in enumerate(weeks_split):
        valid_indexes[i].append(ts.query("week in @weeks_by_fold").index.values)

2380it [04:49,  8.22it/s]


In [47]:
h5f = h5py.File("data/valid_sm_custom1_4fold.h5", "w")
for i,valid_index in enumerate(valid_indexes):
    h5f.create_dataset(f'fold{i}', data=np.concatenate(valid_index))
h5f.close()

***
### custom validation data - by day of year - with shuffle

In [86]:
np.random.seed(23)
n_folds = 4
_train_data = train_data.loc[:, ["building_id","meter","timestamp"]]
ts_uid_values = _train_data.loc[:, ["building_id","meter"]].drop_duplicates()

In [87]:
valid_indexes = [[] for i in range(n_folds)]

for _,row in tqdm(ts_uid_values.iterrows()):
    ts = _train_data.query("building_id == @row.building_id & meter == @row.meter")
    ts["dayofyear"] = ts.timestamp.dt.dayofyear
    days = ts.dayofyear.unique()
    np.random.shuffle(days)
    days_split = np.array_split(days, n_folds)
    np.random.shuffle(days_split)
    
    for i,days_by_fold in enumerate(days_split):
        valid_indexes[i].append(ts.query("dayofyear in @days_by_fold").index.values)

2380it [03:49, 10.35it/s]


In [88]:
h5f = h5py.File("data/valid_sm_custom2_4fold.h5", "w")
for i,valid_index in enumerate(valid_indexes):
    h5f.create_dataset(f'fold{i}', data=np.concatenate(valid_index))
h5f.close()

***
### custom validation data - by day of year - stratified by month - with shuffle

In [81]:
np.random.seed(19)
n_folds = 3
_train_data = train_data.loc[:, ["building_id","meter","timestamp"]]
ts_uid_values = _train_data.loc[:, ["building_id","meter"]].drop_duplicates()

In [82]:
valid_indexes = [[] for i in range(n_folds)]

for _,row in tqdm(ts_uid_values.iterrows()):
    ts = _train_data.query("building_id == @row.building_id & meter == @row.meter")
    ts["dayofyear"] = ts.timestamp.dt.dayofyear
    ts["month"] = ts.timestamp.dt.month
    
    for month in ts.month.unique():
        ts_cut = ts.query("month == @month")
        days = ts_cut.dayofyear.unique()
        np.random.shuffle(days)
        days_split = np.array_split(days, n_folds)
        np.random.shuffle(days_split)
        
        for i,days_by_fold in enumerate(days_split):
            valid_indexes[i].append(ts_cut.query("dayofyear in @days_by_fold").index.values)

2380it [08:46,  4.52it/s]


In [83]:
h5f = h5py.File(f"data/valid_sm_custom_{n_folds}fold.h5", "w")
for i,valid_index in enumerate(valid_indexes):
    h5f.create_dataset(f'fold{i}', data=np.concatenate(valid_index))
h5f.close()

***
### custom validation data - stratified by month - spliting by dayofyear with shuffle

In [68]:
np.random.seed(19)
n_folds = 4
_train_data = train_data.copy(deep=True)
_train_data["year_day"] = _train_data.timestamp.dt.dayofyear
_train_data["month"] = _train_data.timestamp.dt.month

In [69]:
valid_indexes = [[] for i in range(n_folds)]

for month in _train_data.month.unique():
    _train_data_cut = _train_data.query("month == @month")
    days = _train_data_cut.year_day.unique()
    np.random.shuffle(days)
    days_split = np.array_split(days, n_folds)
    np.random.shuffle(days_split)
    
    for i,days_by_fold in enumerate(days_split):
        idx = _train_data_cut.query("year_day in @days_by_fold").index.values
        valid_indexes[i].append(idx)    

In [70]:
h5f = h5py.File(f"data/valid_sm_custom_{n_folds}fold.h5", "w")
for i,valid_index in enumerate(valid_indexes):
    h5f.create_dataset(f'fold{i}', data=np.concatenate(valid_index))
h5f.close()

***
### target scalers

In [27]:
ts_uid_values = (train_data
                 .loc[:, ["building_id","meter"]]
                 .drop_duplicates())
scaling_values = list()

for _,row in tqdm(ts_uid_values.iterrows()):
    ts = train_data.query("building_id == @row.building_id & meter == @row.meter")
    scaler = RobustScaler(with_centering=True, with_scaling=True)
    scaler.fit(ts.meter_reading.values.reshape((-1,1)))
    scaling_values.append((row.building_id, row.meter, scaler.center_[0], scaler.scale_[0]))

2380it [05:03,  7.83it/s]


In [29]:
robust_scaler = pd.DataFrame(scaling_values, columns=["building_id", "meter", "center", "scale"])
robust_scaler.to_csv("data/robust_scaler.csv", index=False)

***